## ExMAS
> Equilibrium matching



Here I:

* generatre demand
* compute groups with ExMAS
* compute costs for groups and travellers
* apply Externality splitting protocol
* run ExMAS
* prune groups with RSIE
* see if there is a group in ExMAS which is then mutually unstable with RSIE - it is (834,29)
* I report why it is unstable


In [1]:
import os
import math
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
cwd = os.getcwd()
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display
pd.options.display.max_columns = None

In [4]:
os.chdir(os.path.join(cwd,'../../..'))
import ExMAS.main
import ExMAS.utils
from ExMAS.utils import inData as inData
from ExMAS.main import matching
from ExMAS.game import games, pricings, prunings, pipeline

In [5]:
params = ExMAS.utils.get_config('ExMAS/spinoffs/game/pipe.json') # load the default 
params.t0 = pd.to_datetime(params.t0)
params.matching_obj = 'u_pax'
inData = ExMAS.utils.load_G(inData, params, stats=True)  # download the graph
params.nP = 100
params.simTime = 0.1
params.shared_discount = 0.3
inData = ExMAS.utils.generate_demand(inData, params)  # generate requests

In [6]:
inData = ExMAS.main(inData, params, plot = False)
KPIs = inData.sblts.res.to_frame('u_pax')
inData = games.prepare_PoA(inData)  # prepare data structures

12-05-22 11:50:46-WARNING-Prepare for game:  1/6
12-05-22 11:50:47-WARNING-Prepare for game:  2/6
12-05-22 11:50:47-WARNING-Prepare for game:  3/6
12-05-22 11:50:47-WARNING-Prepare for game:  4/6
12-05-22 11:50:53-WARNING-Prepare for game:  5/6
12-05-22 11:50:53-WARNING-Prepare for game:  6/6


In [17]:

params.time_cost = params.VoT # travellers' cost per travel time
params.wait_cost = params.time_cost*1.5 # and waiting
params.sharing_penalty_fixed = 0 # fixed penalty (EUR) per 
params.sharing_penalty_multiplier = 0 # fixed penalty (EUR) per 

params.veh_cost = 1.3*params.VoT/params.avg_speed # operating costs per kilometer
params.fixed_ride_cost = 0.5 # ride fixed costs (per vehicle)

In [18]:
inData = games.prepare_PoA(inData)

12-05-22 11:52:45-WARNING-Prepare for game:  1/6
12-05-22 11:52:45-WARNING-Prepare for game:  2/6
12-05-22 11:52:46-WARNING-Prepare for game:  3/6
12-05-22 11:52:46-WARNING-Prepare for game:  4/6
12-05-22 11:52:52-WARNING-Prepare for game:  5/6
12-05-22 11:52:52-WARNING-Prepare for game:  6/6


In [19]:
inData = pricings.update_costs(inData, params)

In [12]:
PRICINGS = dict()  # pricings to apply and their names
PRICINGS['SUBGROUP'] = pricings.subgroup_split
for PRICING, pricing in PRICINGS.items():
    inData = pricing(inData)  # apply pricing strategy

### RSIE 

two *unstable* groups cannot be together in the solution 

if anyone from $G1$ wants to join $G2$ and it is better for the moving traveller and group accepting him.

In [13]:
inData = prunings.algo_EXMAS(inData, price_column='EXTERNALITY')  # apply pruning strategies for a given pricing strategy

In [14]:
inData = pipeline.single_eval(inData, params,
                     MATCHING_OBJS = ['total_group_cost'],  # this can be more
                     PRUNINGS = ['EXMAS'],  # and this can be more
                     PRICING = 'EXTERNALITY',  # this is taken from first level loop
                     minmax = ['min'], EXPERIMENT_NAME = 'debug', store_res = False)  # direction BPoA, WPoAplot_im(inData)

12-05-22 11:52:17-WARNING-Prunings:  ['EXMAS']
12-05-22 11:52:17-WARNING-Pruned nRides 843/843
12-05-22 11:52:17-WARNING-Mutually exclusives 0
12-05-22 11:52:17-WARNING-Experiment-debug_Pricing-EXTERNALITY_Objective-['total_group_cost']_Pruning-['EXMAS']_minmax-min


In [15]:
ExMAS_matching = inData.sblts.rides.selected.to_frame().copy()
ExMAS_matching = ExMAS_matching[ExMAS_matching.selected==1]
selecteds = ExMAS_matching.index

In [16]:
inData = prunings.algo_RSIE(inData, price_column='EXTERNALITY')  # apply pruning strategies for a given pricing strategy

12-05-22 11:52:22-WARNING-Searching unstable pairs 0/843. 0 found so far


KeyError: "['EXTERNALITY'] not in index"

In [14]:
len(inData.sblts.mutually_exclusives_RSIE)

7608

In [18]:
me = inData.sblts.mutually_exclusives_RSIE
for pair in me:
    if pair[0] in selecteds and pair[1] in selecteds:
        print(pair)
        G1 = pair[0]
        G2 = pair[1]
        break


In [63]:
_print = True
price_column = 'EXTERNALITY'
rm = inData.sblts.rides_multi_index  # ride (group) - traveller data
rides = inData.sblts.rides  # rides data
lsuffix = '_x' # suffixes for merges
rsuffix = '_y'
unstables = list()  # output - used as mutual exclusive constrain in ILP
indexes_set = rides.indexes_set # set of travellers for each ride
def are_unstable(G1, G2):
    # see if two groups are unstable
    for i in indexes_set[G1]:  # examine each i in G1
        G2s_with_i = G2s.union({i})  # move i to G2
        for r in rides[rides.indexes_set == G2s_with_i].index:  # loop over rides where i joining G2
            if rm.loc[r, i][price_column] < rm.loc[G1, i][price_column]:  # condition 1 (i want to join G1)
                costs_of_G2_with_i = rm.loc[pd.IndexSlice[r, G2s], :][
                    ['traveller', price_column]]  # costs for travellers in G2 with i
                compare = pd.merge(costs_of_G2, costs_of_G2_with_i, on=['traveller']) # compare prices
                compare['surplus'] = compare[price_column + lsuffix] - compare[price_column + rsuffix] # see which is cheaper
                if compare.surplus.min() >= 0: # if no one is better off
                    if _print: # debugging only
                        print('Group1:', G1, G1s)
                        print('Group2:', G2, G2s)
                        print('Moving traveller:', i)
                        print('Group2 with i:', r, G2s_with_i)
                        print('Costs for i in G1:', rm.loc[G1, i][price_column])
                        print('Costs for i in G2:', rm.loc[r, i][price_column])
                        print('Costs for G2 without i \n ', costs_of_G2[price_column])
                        print('Costs for G2 with i \n ', costs_of_G2_with_i[price_column])
                    return True
    return False

G2s = indexes_set[G2]  # travellers in G2
costs_of_G2 = rm.loc[G2, :][['traveller', price_column]]  # costs of group G2 before joining
G1s = indexes_set[G1]  # travellers in G1
if indexes_set[G1].isdisjoint(indexes_set[G2]):  # if rides are disjoint
    are_unstable(G1, G2)

In [56]:
rm[rm.ride == 834][['ttrav_sh','delay','cost_user','EXTERNALITY']]

ttrav_sh  delay  cost_user  EXTERNALITY
834 1        755   31.0    2.80525      4.42435
    49       616  -16.0    2.24000      2.25855
    78       680  -25.0    2.51125      3.66695

In [57]:
rm[rm.ride == 29][['ttrav_sh','delay','cost_user','EXTERNALITY']]

,,ttrav_sh,delay,cost_user,EXTERNALITY
29,29,148,0.0,0.518,1.6914


In [62]:
rm[rm.ride == 295][['ttrav_sh','delay','cost_user','EXTERNALITY']]

ttrav_sh  delay  cost_user  EXTERNALITY
295 29       178   20.5   0.730625      1.26805
    49       521   -5.5   1.852375      4.05335

---
(c) Rafał Kucharski, Delft, 2021